## Recommendation System Based on  Director (Director's Name taken as User Input)

### Importing the Required Libraries & Files

In [9]:
import pandas as pd
import numpy as np
import sys

movies= pd.read_csv("movies.dat",encoding='latin-1',sep='\t')
movies.drop(['imdbID','spanishTitle','rtID','rtID', 'rtAllCriticsRating', 'rtAllCriticsNumReviews',\
       'rtAllCriticsNumFresh', 'rtAllCriticsNumRotten', 'rtAllCriticsScore',\
       'rtTopCriticsRating', 'rtTopCriticsNumReviews', 'rtTopCriticsNumFresh',\
       'rtTopCriticsNumRotten', 'rtTopCriticsScore',\
       'rtAudienceNumRatings', 'rtAudienceScore', 'rtPictureURL'],axis=1,inplace=True)
movies['title'].loc[0]='Toy Story'

movie_genre= pd.read_csv("movie_genres.dat",encoding='latin-1',sep='\t')
movie_genre = movie_genre.groupby('movieID').agg({ 'genre': ', '.join}).reset_index()

movie_director= pd.read_csv('movie_directors.dat',encoding='latin-1',sep='\t')

movie_director.drop(['directorID'],axis=1,inplace=True)

movie_actor= pd.read_csv('movie_actors.dat',encoding='latin-1',sep='\t')
movie_actor.drop(['actorID'],axis=1,inplace=True)

merge1= pd.merge(movies,movie_genre,left_on='id',right_on='movieID')
merge2= pd.merge(merge1,movie_director)
merge3= pd.merge(merge2,movie_actor)
merge3.drop(['movieID'],axis=1,inplace=True)

user_ratedmovies= pd.read_csv('user_ratedmovies.dat',encoding='latin-1',sep='\t')
user_ratedmovies.drop(['date_day', 'date_month', 'date_year',\
       'date_hour', 'date_minute', 'date_second'],axis=1,inplace=True)

final_data= pd.merge(merge3,user_ratedmovies,left_on='id',right_on='movieID')

### Taking Director's Name as User Input

In [10]:
dir_needed= str(input("Enter the Full Name of the Director: "))

Enter the Full Name of the Director: John Woo


In [11]:
### Finding all the Movies directed by Director
dir_mov= movie_director[movie_director['directorName']==dir_needed]

### Checking if Director is present in the database or not
if len(dir_mov)>=1:
    print("Director Found")
else:
    print("Director Not Found in Database")
    sys.exit("There is no Director by this Name, please re-check")

Director Found


### Finding all the movies which has been watched by the User 

In [12]:
user_ratedmovies.groupby(['userID','movieID']).count()
user= int(input("Enter your User ID : "))

Enter your User ID : 78


In [13]:
subset_user_ratedmovies= user_ratedmovies[user_ratedmovies['userID']==user]
### Checking if User is present in the database or not
if len(subset_user_ratedmovies)>=1:
    print("User Found")
else:
    print("Invalid User ID, please re-check")
    sys.exit("User ID Not Found in Database")

User Found


In [14]:
### Recommending Movies of Director which has not been watched by User Yet

df_all = dir_mov.merge(subset_user_ratedmovies.drop_duplicates(), on=['movieID'],how='left', indicator=True)
recc1= df_all[df_all['_merge']=='left_only']
recc_movie= recc1[['movieID','directorName']]
final_recc= pd.merge(recc_movie,movies,left_on='movieID',right_on='id',how='left')
final_recc.drop(['id'],inplace=True,axis=1)

### List of Movies of the Director which has not been watched by User Yet
final_recc= final_recc.sort_values(by='rtAudienceRating',ascending=False)
final_recc.drop_duplicates('title',inplace=True,keep='first')
### Top 5 Movie Recommendations
final_recc.head(5)

,movieID,directorName,title,imdbPictureURL,year,rtAudienceRating
4,1218,John Woo,Dip huet seung hung,http://ia.media-imdb.com/images/M/MV5BMTIzNjAy...,1989,4
3,702,John Woo,Face/Off,http://ia.media-imdb.com/images/M/MV5BMTQ3MDY4...,1997,3.5
6,5420,John Woo,Windtalkers,http://ia.media-imdb.com/images/M/MV5BODY3OTI4...,2002,3.2
0,95,John Woo,Broken Arrow,http://ia.media-imdb.com/images/M/MV5BMjE0NjIw...,1996,2.9
1,139,John Woo,Hard Target,http://ia.media-imdb.com/images/M/MV5BODQxMzY3...,1993,0
